In [1]:
import pickle
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics.pairwise import cosine_similarity
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import numpy as np
from scipy.spatial import distance
import warnings
warnings.filterwarnings("ignore")

Classe predittore: 
- predizione_ultima_giornata_non_anomala: 
prende come **input** una riga dal file dati_riorganizzati che rappresenta un dataframe nel nuovo formato composto da tutte le normali prima di un'unica giornata anomala e un contatore utile per salvare l'immagine di output. 
Come **output** fornisce il confronto visivo  e la cosine distance tra la predizione effettuata su tutte le giornati normali precedenti all'ultima (normale) e quest'ultima.

- predizione_giornata_anomala: stessi **input** del metodo predizione_ultima_giornata_non_anomala ma questa volta si shifta di una giornata, non considerando quindi la prima giornata normala e includendo l'ultima giornata normale. L'**output** è un confronto visivo e la cosine distance tra la predizione e la giornata anomala.


In [2]:
class Predittore:

  def __init__(self,path):
    self.path = path
    
  def predizione_ultima_giornata_non_anomala(self, dato, cont):

    # per predire ultima giornata non anomala uno i valori che vanno da 0 a [(indice_giorno_anomalo - 1) - 23]
    df_senza_anomalie = dato[dato["Anomalous"] == 0]
    df_target = df_senza_anomalie[0:-23] 
    df_da_predire = df_senza_anomalie[-23:]
    #df_da_predire.to_csv("dapredire.csv")
    #df_target.to_csv("target.csv")

    n_input = 23
    n_features = 1

    del df_target["Index"]
    del df_target["Anomalous"]
    del df_target["Finestra"]

    train = df_target
    scaler = MinMaxScaler()
    scaler.fit(train)
    train = scaler.transform(train)

    # parametri: batch_size=32, epochs = 100
    generator = TimeseriesGenerator(train, train, length=n_input, batch_size=32)
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
    model.add(Dropout(0.15))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(generator, epochs=90)

    pred_list = []
    batch = train[-n_input:].reshape((1, n_input, n_features))

    for i in range(n_input):   
      pred_list.append(model.predict(batch)[0]) 
      batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

    df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), index=df_senza_anomalie[-n_input:].index, columns=['Prediction'])

    plt.figure(figsize=(40,20))
    plt.title("Confronto tra la giornata predetta e la giornata normale")
    plt.plot(df_predict.index, df_predict["Prediction"], color='r', marker="o")
    plt.plot(df_da_predire.index,df_da_predire["Affluenza"],color = "b",marker="o")
    plt.legend(loc='best', fontsize='small')
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=18)
    plt.xticks(rotation=70)

    plt.savefig('confronto_normale' + str(cont) + '.png', bbox_inches='tight')

    print(distance.cosine(df_predict["Prediction"],df_da_predire["Affluenza"]))


  def predizione_giornata_anomala(self, dato, cont):
  
    # per predire la gioranta anomala dobbiamo prendere le giornate che vanno dalla seconda all'ultima normale
    df_senza_anomalie = dato[dato["Anomalous"] == 0]
    df_giornata_anomala = dato[dato["Anomalous"] == 1]

    # tolgo la prima giornata
    df_target = df_senza_anomalie[:23]

    df_da_predire = df_senza_anomalie.drop(df_target.index)
    # df_da_predire.to_csv("senza_anomalie.csv")
    
    n_input = 23
    n_features = 1

    del df_da_predire["Index"]
    del df_da_predire["Anomalous"]
    del df_da_predire["Finestra"]

    train = df_da_predire
    scaler = MinMaxScaler()
    scaler.fit(train)
    train = scaler.transform(train)

    # parametri: batch_size=32, epochs = 100
    generator = TimeseriesGenerator(train, train, length=n_input, batch_size=32)
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
    model.add(Dropout(0.15))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(generator, epochs=90)

    pred_list = []
    batch = train[-n_input:].reshape((1, n_input, n_features))

    for i in range(n_input):   
      pred_list.append(model.predict(batch)[0]) 
      batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

    df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), index=df_da_predire[-n_input:].index, columns=['Prediction'])

    plt.figure(figsize=(40,20))
    plt.title("Confronto tra la giornata predetta e la giornata anomala")
    plt.plot(df_giornata_anomala.index, df_predict["Prediction"], color='r', marker="o")
    plt.plot(df_giornata_anomala.index,df_giornata_anomala["Affluenza"],color = "b",marker="o")
    plt.legend(loc='best', fontsize='small')
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=18)
    plt.xticks(rotation=70)

    plt.savefig('confronto_anomalia' + str(cont) + '.png', bbox_inches='tight')

    print(distance.cosine(df_predict["Prediction"],df_giornata_anomala["Affluenza"]))


Semplice main che scorre i dati e li passa, un valore alla volta, al predittore.

In [ ]:
p = Predittore("dati_organizzati.pkl")

with open("dati_riorganizzati.pkl","rb") as f:
  df_list = pickle.load(f)
  cont = 0

  for i in range(len(df_list)):
    dato = df_list[i]

    # 22 è il numero medio di giorni consecutivi normali, valore che usiamo come soglia per la dimensione minima della finestra
    if dato["Finestra"].max() > 22*23:
      p.predizione_ultima_giornata_non_anomala(dato,cont)
      p.predizione_giornata_anomala(dato,cont)

      cont += 1

Epoch 1/90
39/39 [==============================] - 2s 43ms/step - loss: 0.1216
Epoch 2/90
39/39 [==============================] - 2s 44ms/step - loss: 0.0538
Epoch 3/90
39/39 [==============================] - 2s 44ms/step - loss: 0.0341
Epoch 4/90
39/39 [==============================] - 2s 42ms/step - loss: 0.0287
Epoch 5/90
39/39 [==============================] - 2s 41ms/step - loss: 0.0259
Epoch 6/90
39/39 [==============================] - 2s 42ms/step - loss: 0.0244
Epoch 7/90
39/39 [==============================] - 2s 43ms/step - loss: 0.0256
Epoch 8/90
39/39 [==============================] - 2s 42ms/step - loss: 0.0205
Epoch 9/90
39/39 [==============================] - 2s 41ms/step - loss: 0.0183
Epoch 10/90
39/39 [==============================] - 2s 42ms/step - loss: 0.0173
Epoch 11/90
39/39 [==============================] - 2s 47ms/step - loss: 0.0179
Epoch 12/90
39/39 [==============================] - 2s 45ms/step - loss: 0.0169
Epoch 13/90
39/39 [==================